In [1]:
import pandas as pd
from sklearn import tree
import pydotplus
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
import numpy as np

In [2]:
# Import the datasets 
senseData = pd.read_csv("Lerror2.csv", header = None)
senseData.columns = ["Case_ID", "Timestamp", "Type", "Sensor"]
senseData

,Case_ID,Timestamp,Type,Sensor
0,2022-09-02,2022-09-02T08:10:52.403228000,PresenceSensor,Door_Y
1,2022-09-02,2022-09-02T08:10:53.610334781,FloorSensor,Sens3_Y
2,2022-09-02,2022-09-02T08:13:21.438761905,FloorSensor,Sens7_Y
3,2022-09-02,2022-09-02T08:13:24.267189029,FloorSensor,Sens5_Y
4,2022-09-02,2022-09-02T08:13:29.302722934,FloorSensor,Sens1_Y
...,...,...,...,...
1564,2022-12-10,2022-12-10T08:33:28.078615897,FloorSensor,Sens1_X
1565,2022-12-10,2022-12-10T08:35:29.699936240,FloorSensor,Sens2_X
1566,2022-12-10,2022-12-10T08:35:31.699936240,FloorSensor,Sens4_X
1567,2022-12-10,2022-12-10T08:35:33.699936240,FloorSensor,Sens6_X


In [3]:
newdataset = pd.DataFrame(columns =["Case_ID", "Timestamp", "Type", "Sensor", "Label", "Activity_ID"])
temp = 0
i = 0

for index, row in senseData.iterrows():
    activity = row["Sensor"][-1]
    new_row = {}
    new_row["Case_ID"] = row["Case_ID"]
    new_row["Timestamp"] = row["Timestamp"]
    new_row["Type"] = row["Type"]
    new_row["Sensor"] = row["Sensor"]
    # First row of the dataset
    if(temp == 0 and i == 0):
        new_row["Label"] = activity
        new_row["Activity_ID"] = i
        temp = activity
    # if the actual row has the same label of the stored, then is the same group
    elif(activity == temp):
        new_row["Activity_ID"] = i
        new_row["Label"] = activity
    # if the label mismatch, start a new group
    elif(activity != temp):
        i = i+1
        temp = activity
        new_row["Activity_ID"] = i
        new_row["Label"] = activity
    newdataset = newdataset.append(new_row, ignore_index=True)
newdataset

,Case_ID,Timestamp,Type,Sensor,Label,Activity_ID
0,2022-09-02,2022-09-02T08:10:52.403228000,PresenceSensor,Door_Y,Y,0
1,2022-09-02,2022-09-02T08:10:53.610334781,FloorSensor,Sens3_Y,Y,0
2,2022-09-02,2022-09-02T08:13:21.438761905,FloorSensor,Sens7_Y,Y,0
3,2022-09-02,2022-09-02T08:13:24.267189029,FloorSensor,Sens5_Y,Y,0
4,2022-09-02,2022-09-02T08:13:29.302722934,FloorSensor,Sens1_Y,Y,0
...,...,...,...,...,...,...
1564,2022-12-10,2022-12-10T08:33:28.078615897,FloorSensor,Sens1_X,X,143
1565,2022-12-10,2022-12-10T08:35:29.699936240,FloorSensor,Sens2_X,X,143
1566,2022-12-10,2022-12-10T08:35:31.699936240,FloorSensor,Sens4_X,X,143
1567,2022-12-10,2022-12-10T08:35:33.699936240,FloorSensor,Sens6_X,X,143


In [4]:
# Convert the date columns in datetime type
newdataset['Timestamp'] = pd.to_datetime(newdataset['Timestamp'], format="%Y-%m-%dT%H:%M:%S.%f")
newdataset

,Case_ID,Timestamp,Type,Sensor,Label,Activity_ID
0,2022-09-02,2022-09-02 08:10:52.403228000,PresenceSensor,Door_Y,Y,0
1,2022-09-02,2022-09-02 08:10:53.610334781,FloorSensor,Sens3_Y,Y,0
2,2022-09-02,2022-09-02 08:13:21.438761905,FloorSensor,Sens7_Y,Y,0
3,2022-09-02,2022-09-02 08:13:24.267189029,FloorSensor,Sens5_Y,Y,0
4,2022-09-02,2022-09-02 08:13:29.302722934,FloorSensor,Sens1_Y,Y,0
...,...,...,...,...,...,...
1564,2022-12-10,2022-12-10 08:33:28.078615897,FloorSensor,Sens1_X,X,143
1565,2022-12-10,2022-12-10 08:35:29.699936240,FloorSensor,Sens2_X,X,143
1566,2022-12-10,2022-12-10 08:35:31.699936240,FloorSensor,Sens4_X,X,143
1567,2022-12-10,2022-12-10 08:35:33.699936240,FloorSensor,Sens6_X,X,143


In [5]:
out = pd.DataFrame(columns = ["Case_ID", "Timestamp", "Type", "Sensor", "Label", "Activity_ID"])
i = 0
for idc, case in newdataset.groupby("Case_ID"):
    for id, window in case.groupby(pd.Grouper(freq="30s", key = "Timestamp")):
        if(window.empty):
            pass
        else:
            most_freq_label = window['Label'].value_counts().idxmax()
            dict = {"Case_ID":[],"Timestamp":[],"Type":[],"Sensor":[],"Label":[],"Activity_ID":[]}
            for a, el in window.iterrows():
                dict.get("Case_ID").append(el.get("Case_ID"))
                dict.get("Timestamp").append(el.get("Timestamp"))
                dict.get("Type").append(el.get("Type"))
                dict.get("Sensor").append(el.get("Sensor"))
                dict.get("Label").append(most_freq_label)
                dict.get("Activity_ID").append(i)
            i = i+1
            df2 = pd.DataFrame(dict)
            out = pd.concat([out, df2], ignore_index = True)
            # if(i==20):
            #     break

In [6]:
out

,Case_ID,Timestamp,Type,Sensor,Label,Activity_ID
0,2022-09-02,2022-09-02 08:10:52.403228000,PresenceSensor,Door_Y,Y,0
1,2022-09-02,2022-09-02 08:10:53.610334781,FloorSensor,Sens3_Y,Y,0
2,2022-09-02,2022-09-02 08:13:21.438761905,FloorSensor,Sens7_Y,Y,1
3,2022-09-02,2022-09-02 08:13:24.267189029,FloorSensor,Sens5_Y,Y,1
4,2022-09-02,2022-09-02 08:13:29.302722934,FloorSensor,Sens1_Y,Y,1
...,...,...,...,...,...,...
1564,2022-12-10,2022-12-10 08:33:28.078615897,FloorSensor,Sens1_X,X,728
1565,2022-12-10,2022-12-10 08:35:29.699936240,FloorSensor,Sens2_X,X,729
1566,2022-12-10,2022-12-10 08:35:31.699936240,FloorSensor,Sens4_X,X,730
1567,2022-12-10,2022-12-10 08:35:33.699936240,FloorSensor,Sens6_X,X,730


In [7]:
# Save the file
out.to_csv("Lerror2_labeled.csv", index=False)